In [2]:
import psycopg2
database = "rssfeed"
hostname="rssfeed.cjgj2uy1bapa.us-east-1.rds.amazonaws.com"
port="5432" 
userid="postgres"
passwrd=""
conn_string = "host="+hostname+" port="+port+" dbname="+database+" user="+userid+" password="+passwrd
conn = psycopg2.connect(conn_string)
conn.autocommit=True
cursor = conn.cursor()
sqlSelect = "SELECT * FROM rss_entities_with_pub";
cursor.execute(sqlSelect);
rows = cursor.fetchall();

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
## Publisher url codes
import pandas as pd
from tqdm import tqdm
publishers = pd.read_json('data/rss_feed.json', typ='series')
publishers = publishers.to_dict()
pub = {v: k for k, v in publishers.items()}

In [ ]:
rssdf = pd.DataFrame(rows)
rssdf.columns = ['publisher_code','entity','publish_time']
rssdf["publisher"] = rssdf["publisher_code"].map(pub)  
rssdf.head()

,publisher_code,entity,publish_time,publisher
0,12787,Ministry of External Affairs,2020-02-06T17:05:24+05:30,https://www.thehindubusinessline.com/markets/c...
1,11750,Jay-Z,2020-02-06T21:28:13+00:00,https://www.huffingtonpost.com.au/feeds/index.xml
2,11750,Kobe Bryant,2020-02-06T21:28:13+00:00,https://www.huffingtonpost.com.au/feeds/index.xml
3,11750,Giannas,2020-02-06T21:28:13+00:00,https://www.huffingtonpost.com.au/feeds/index.xml
4,11750,Columbia University,2020-02-06T21:28:13+00:00,https://www.huffingtonpost.com.au/feeds/index.xml


In [ ]:
from datetime import datetime
rssdf['24hour_bucket'] = pd.to_datetime(rssdf['publish_time'],errors='coerce',format = '%Y-%m-%dT%H:%M:%S+00:00',\
                                        infer_datetime_format = True, cache = True)

print(rssdf['publish_time'][0])


2020-02-06T17:05:24+05:30


In [ ]:
rssdf.head()
rssdf['dates']= rssdf['24hour_bucket'].dt.date
rssdf["dates"] = pd.to_datetime(rssdf['dates'])
rssdf.head(3)


,publisher_code,entity,publish_time,publisher,24hour_bucket,dates
0,12787,Ministry of External Affairs,2020-02-06T17:05:24+05:30,https://www.thehindubusinessline.com/markets/c...,NaT,NaT
1,11750,Jay-Z,2020-02-06T21:28:13+00:00,https://www.huffingtonpost.com.au/feeds/index.xml,2020-02-06 21:28:13,2020-02-06
2,11750,Kobe Bryant,2020-02-06T21:28:13+00:00,https://www.huffingtonpost.com.au/feeds/index.xml,2020-02-06 21:28:13,2020-02-06


## Generate Dataset


In [ ]:
import datetime
import numpy as np
newDf = pd.DataFrame(rssdf.loc[:, ["entity", "dates"]])
newDf = newDf.sort_values(by = "dates")
newDf = newDf.reset_index(drop=True)
newDf.head(3)

,entity,dates
0,London,1970-01-01
1,British,1970-01-01
2,Kevin Pollak's,1970-07-02


In [ ]:
# selecting date range
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

newDf = newDf[ newDf.dates > pd.datetime(2017,1, 1)]
newDf = newDf[newDf.dates<pd.datetime(2020,3,8)]
newDf = newDf.reset_index(drop=True)
newDf.head(3) 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  import sys


ERROR! Session/line number was not unique in database. History logging moved to new session 59


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  


,entity,dates
0,PT Money,2017-01-02
1,New Years Eve,2017-01-02
2,Instagram,2017-01-02


In [ ]:
import datetime as dt
time_range = pd.date_range(start = "2017-01-01", end = "2020-03-08", freq = "D").to_series()
list_range = list(zip(time_range.dt.strftime('%Y-%m-%d'), time_range.shift(-4).dropna().dt.strftime('%Y-%m-%d')))
list_range[-1][1]

'2020-03-08'

## Summary

Thoughts: For each row (7-day period), I have a collection of entities and rank they by their frequencies in each slicing window.

Rank an entity by its frequency over the entire dataset and the frequency amony the slicing windows and its frequency in its 7-day time window, then assign some weight to each time window. (Simliar to TF-IDF)

1.   Score for slicing window: 30 - 1
2.   Score for entity in each slicing window: 10 - 1
3.   Weight for each entity in all slicing window: number of entity  appeared times / number of total entity appears times

Final score for the entity: (part_1 + part_2) * part_3










## Generate top 10 high frequency entity for each slicing window



In [ ]:
from collections import defaultdict
import operator
def generate_entity(df, list_range):
    slicing_window = []
    time_index = 0
    start_range = [dt.datetime.strptime(list_range[0][0], '%Y-%m-%d').date(), dt.datetime.strptime(list_range[0][1], '%Y-%m-%d').date()]
    dic = defaultdict(int)
    total_dict = defaultdict(int)
    total_entity = 0
    #max_freq = 0
    for i in tqdm(range(df.shape[0])):
        time = df.loc[i,"dates"]
        if time > start_range[1]:
            dic = dict(sorted(dic.items(), key=operator.itemgetter(1),reverse=True))
            #   max_freq = max(len(dic.keys() ), max_freq)
            slicing_window.append(dic )
            dic = defaultdict(int)
            time_index += 1
            start_range = [dt.datetime.strptime(list_range[time_index][0], '%Y-%m-%d').date(), dt.datetime.strptime(list_range[time_index ][1], '%Y-%m-%d').date()]
        #max_freq = max(len(dic.keys() ), max_freq)
        item =  df.loc[i, "entity"]
        total_dict[item] += 1
        dic[item] += 1
        total_entity += 1
    total_dict = dict(sorted(total_dict.items(), key=operator.itemgetter(1),reverse=True))
    #generate_df = pd.DataFrame(data = None, columns = ["Time_range"] + ["Entity" + str(i) for i in range(1, max_freq + 1)])
    generate_df = pd.DataFrame(data = None, columns = ["Time_range"] + ["Entity" + str(i) for i in range(1, 11)])
    generate_df["Time_range"] = list_range
    for i in tqdm(range(len(slicing_window))):
        temp = slicing_window[i]
        #generate_df.loc[i,1:]  = [ i for i in temp.keys()]  + [None] * (generate_df.shape[1] - 1 - len(list(temp.keys())))
        temp = list(temp.keys())
        if len(temp) < 10:
            temp += [None] * (10 -len(temp))
        generate_df.loc[i,1:] = temp[:10]
    return generate_df , total_dict, total_entity
    

In [ ]:
entityCollection, totalDict, countEntity = generate_entity(newDf, list_range)

100%|██████████| 1157/1157 [00:00<00:00, 2940.28it/s]


In [ ]:
entityCollection.head()

,Time_range,Entity1,Entity2,Entity3,Entity4,Entity5,Entity6,Entity7,Entity8,Entity9,Entity10
0,"(2017-01-01, 2017-01-05)",New Year,Gizele Oliveira,Facebook,Twitter,Thorsten Walles,iTunes,Heike Walles,US,Patreon,English
1,"(2017-01-02, 2017-01-06)",Facebook,Twitter,iTunes,Shira Weinberger Photography,Kalen Bruce,Freebie Podcast Series,Instagram,Leo Babauta,Trump,Brian Altano
2,"(2017-01-03, 2017-01-07)",Jenny Craig,Surface Studio,Scott Anfang,Rancho Obi-Wan,IP,Dell,Christopher Mott,Eileen Opatut,Japanese,Terrible Board
3,"(2017-01-04, 2017-01-08)",Tii - iTem,Gregory Koldirkaev,Jeff Kear,gespeeld,The Live Fit Girls,Bret Bonnet,Obamacare,Digital Dating Disruptor,The Feed Podcast App Store Shatters Records,New Year
4,"(2017-01-05, 2017-01-09)",Guru Singh,iTunes,Twitter,Brittany Hughes,Haben Girma,Patreon,Facebook,Donald Trump,Le Chiffre,English


In [ ]:
entityCollection.iloc[0,1]
countEntity

6166851

## Scoring the entity from the recent month

In [ ]:
from sklearn import preprocessing

def scoring(totalDict, number,entityCollection, countEntity):
    unique_entity = defaultdict(int)
    current_slice = 1/30
    for i in tqdm(range(len(entityCollection) - 7*30, len(entityCollection))):
        for j in range(1, 11):
            entity = entityCollection.iloc[i, j]
            if entity:
                current_score = (11 - j + current_slice)
                current_slice += 1/30
                unique_entity[entity] = max(unique_entity[entity], current_score)
      
    for key in list(unique_entity.keys())[:-1]:
        new_score = unique_entity[key] * totalDict[key] / countEntity
        unique_entity[key] = new_score

    df = pd.DataFrame.from_dict(unique_entity, orient='index')
    df = df[:-1].reset_index()
    df.columns = ["Entity", "Score"]
    min_max_scaler = preprocessing.MinMaxScaler()
    x = df[['Score']].values.astype(float)
    x_scaled = min_max_scaler.fit_transform(x)
    df_normalized = pd.DataFrame(x_scaled).apply(lambda x: round(x * 100,  2))
    df[["Score"]] = df_normalized
    df = df.sort_values('Score', ascending= False).reset_index().drop("index", axis = 1)
    return df


entity_rank = scoring(totalDict, 30, entityCollection, countEntity)

entity_rank.head(30)

100%|██████████| 210/210 [00:00<00:00, 10067.71it/s]


,Entity,Score
0,UK,100.00
1,US,96.32
2,China,93.10
3,Trump,83.06
4,India,61.79
5,U.S.,61.36
6,London,52.78
7,Facebook,49.59
8,Instagram,43.64
9,Twitter,40.06


In [ ]:
events_df = pd.read_csv('events_df.csv')

In [ ]:
events_df.head(10)

,Unnamed: 0,features,start,end,index_tuple
0,0,"['U.S.', 'Mercedes', '', ""International Women'...",2020-03-01,2020-03-07,"[(0, 0), (0, 1), (0, 2)]"
1,0,"['Australia', 'Australian', 'Sydney']",2020-02-10,2020-02-10,5
2,1,"['California', 'Florida', 'New York', 'Texas']",2020-02-10,2020-02-10,8
3,2,"['Amazon', 'Apple', 'Google']",2020-02-11,2020-02-11,2
4,3,"['', 'Amazon', 'Facebook', 'Google', 'Instagra...",2020-02-14,2020-02-14,1
5,4,"['Aston Villa', 'Juventus', 'Man City']",2020-02-16,2020-02-16,2
6,5,"['Iran', 'Russia', 'Russian']",2020-02-24,2020-02-24,2
7,6,"['European', 'France', 'French', 'German', 'Ge...",2020-02-24,2020-02-24,5
8,7,"['California', 'Chicago', 'Florida', 'New York...",2020-02-24,2020-02-24,7
9,8,"['Amazon', 'Apple', 'Google']",2020-02-24,2020-02-24,8


In [ ]:
#@title Mapping entity to events
import ast
def generate_event_dic(events_df):
    """
    Generate a dictionary which map entity to the event.
    Assume no overlapping entity among these events.
    """
    event_collection = events_df.features.unique()
    event_map = {}
    for events in event_collection:
        event = ast.literal_eval(events)
        for entity in event:
            if entity: 
                event_map[entity] = events
    return event_map
event_dic = generate_event_dic(events_df)

In [ ]:
def generate_event(entity_rank, event_dic):
    event = []
    for i in range(entity_rank.shape[0]):
        if entity_rank.iloc[i, 0] in event_dic:
            event.append(event_dic[entity_rank.iloc[i, 0]])
        else:
            event.append(entity_rank.iloc[i, 0])
        entity_rank["Event"] = event
    return entity_rank

entity_rank2 = generate_event(entity_rank, event_dic)

In [ ]:
entity_rank2.head(20)

,Entity,Score,Event
0,UK,100.00,"['U.S.', 'Mercedes', '', ""International Women'..."
1,US,96.32,"['U.S.', 'Mercedes', '', ""International Women'..."
2,China,93.10,"['U.S.', 'Mercedes', '', ""International Women'..."
3,Trump,83.06,"['U.S.', 'Mercedes', '', ""International Women'..."
4,India,61.79,"['U.S.', 'Mercedes', '', ""International Women'..."
5,U.S.,61.36,"['U.S.', 'Mercedes', '', ""International Women'..."
6,London,52.78,"['U.S.', 'Mercedes', '', ""International Women'..."
7,Facebook,49.59,"['', 'Amazon', 'Apple', 'Facebook', 'Google', ..."
8,Instagram,43.64,"['', 'Amazon', 'Apple', 'Facebook', 'Google', ..."
9,Twitter,40.06,"['', 'Amazon', 'Apple', 'Facebook', 'Google', ..."


## Improve
This is a simple but efficient approach to socre the entities. Here we can see it really catched some current event such as coronavirus and Kobe Bryant. Here we want to capture the event through entities, we should find some way to combine the most frequncy entity to more specifc event.